<a href="https://colab.research.google.com/github/yukinaga/brain_ai_book/blob/master/neural_network_on_torus_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **恒常性の導入**
今回のネットワークでは、「入試」方式によりニューロンの合格/不合格、すなわち興奮/抑制を決めることでネットワークに恒常性を導入します。

* 活性化関数への入力が大きい順にニューロンを並べ、上位を一定の割合で選別し出力を1（興奮）とする
* 残りのニューロンの出力は0（抑制）とする

これにより、常に一定の割合でニューロンが興奮することになり、ネットワークの恒常性が保たれることになります。


In [ ]:
# import numpy as np  # CPU
import cupy as np  # GPU
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc

# @markdown ### **トーラス上のニューラルネットワーク**
# @markdown 投射ニューロンの割合を入力してください。
proj_ratio = 0.5 # @param {type:"number"}

# @markdown 抑制性ニューロンの割合を入力してください。
inhib_ratio = 0.2 # @param {type:"number"}

# @markdown タイムステップ数を入力してください。
steps = 240 # @param {type:"number"}

# @markdown ### **恒常性の導入**
# @markdown 興奮させるニューロンの割合を入力してください。
excite_ratio = 0.5  # @param {type:"number"}

# @markdown ### **動画の保存**
# @markdown GIFアニメーションを保存しますか？
save_gif = False #@param {type:"boolean"}
# @markdown MP4形式の動画を保存しますか？
save_mp4 = False #@param {type:"boolean"}

n_h = 256  # 領域高さ
n_w = 256  # 領域幅
n_connect = 64  # ニューロンへの入力数
sigma_inter = 4  # 介在ニューロンの軸索長の標準偏差
w_mu = 0.25  # 重みの平均値 
w_sigma = 0.08  # 重みの標準偏差

matplotlib.rcParams["animation.embed_limit"] = 2**128  # アニメーションのサイズの上限

# トラース上のニューラルネットワークのクラス
class TorusNetwork():
    def __init__(self, n_h, n_w, n_connect):
        n_neuron = n_h * n_w  # ニューロンの総数
        self.params = (n_h, n_w, n_neuron, n_connect)

        self.connect_ids = None  # シナプス前細胞のインデックス 
        self.w = None  # 重み
        self.b = None  # バイアス
        self.y = None  # ニューロンの出力
        self.proj = None  # 投射ニューロンかどうか
        self.proj_ids = None  # 投射ニューロンのインデックス
        self.proj_to_ids = None  # 投射先のインデックス
        self.inhib = None  # 抑制性ニューロンかどうか

    def connect(self, proj_ratio, sigma_inter):
        n_h, n_w, n_neuron, n_connect = self.params
        
        # 投射ニューロンをランダムに選択
        n_proj= int(proj_ratio * n_neuron)
        rand_ids = np.random.permutation(np.arange(n_neuron))
        self.proj_ids = rand_ids[:n_proj]
        self.proj_to_ids = np.random.permutation(self.proj_ids)
        self.proj = np.zeros(n_neuron, dtype=np.bool)
        self.proj[self.proj_ids] = True

        # 介在ニューロンのx座標     
        inter_dist_x = np.random.randn(n_neuron, n_connect) * sigma_inter
        inter_dist_x = np.where(inter_dist_x<0, inter_dist_x-0.5, inter_dist_x+0.5).astype(np.int32) 
        x_connect = np.zeros((n_neuron, n_connect), dtype=np.int32)
        x_connect += np.arange(n_neuron).reshape(-1, 1)
        x_connect %= n_w
        x_connect += inter_dist_x
        x_connect = np.where(x_connect<0, x_connect+n_w, x_connect)
        x_connect = np.where(x_connect>=n_w, x_connect-n_w, x_connect)

        # 介在ニューロンのy座標        
        inter_dist_y = np.random.randn(n_neuron, n_connect) * sigma_inter
        inter_dist_y = np.where(inter_dist_y<0, inter_dist_y-0.5, inter_dist_y+0.5).astype(np.int32)        
        y_connect = np.zeros((n_neuron, n_connect), dtype=np.int32)
        y_connect += np.arange(n_neuron).reshape(-1, 1)
        y_connect //= n_w
        y_connect += inter_dist_y
        y_connect = np.where(y_connect<0, y_connect+n_h, y_connect)
        y_connect = np.where(y_connect>=n_h, y_connect-n_h, y_connect)        

          # シナプス前細胞のインデックス
        self.connect_ids = x_connect + n_w * y_connect
        
    def initialize_network(self, inhib_ratio, w_mu, w_sigma):
        n_h, n_w, n_neuron, n_connect = self.params

        # 抑制性ニューロンをランダムに選択
        n_inhib = int(inhib_ratio * n_neuron)
        rand_ids = np.random.permutation(np.arange(n_neuron))
        inhib_ids = rand_ids[:n_inhib]
        self.inhib = np.zeros(n_neuron, dtype=np.bool)
        self.inhib[inhib_ids] = True
        
        # 重みとバイアスの初期化
        self.w = np.random.randn(n_neuron, n_connect) * w_sigma + w_mu
        self.w = np.where(self.inhib[self.connect_ids], -self.w, self.w)
        self.w /= np.sum(self.w, axis=1, keepdims=True)
        self.b = np.zeros(n_neuron)
        
        # 出力を初期化
        self.y = np.random.randint(0, 2, n_neuron)

    def forward(self, excite_ratio):
        n_h, n_w, n_neuron, n_connect = self.params
        n_excite = int(excite_ratio * n_neuron)  # 興奮したニューロン数

        # ニューラルネットワークの計算
        self.y[self.proj_to_ids] = self.y[self.proj_ids]  # 投射
        x = self.y[self.connect_ids]
        u = np.sum(self.w*x, axis=1) - self.b
        larger_ids = np.argpartition(-u, n_excite)[:n_excite]  # 恒常性
        self.y[:] = 0
        self.y[larger_ids] = 1

# ネットワークの設定
tnet = TorusNetwork(n_h, n_w, n_connect)
tnet.connect(proj_ratio, sigma_inter)
tnet.initialize_network(inhib_ratio, w_mu, w_sigma)

# 結果表示のための設定
figure = plt.figure()
plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
plt.tick_params(bottom=False, left=False, right=False, top=False)

# 興奮したニューロンのカラー
c_excite = np.array([30, 144, 255]).reshape(1, -1)
c_map = np.zeros((n_h*n_w, 3)) + c_excite

# 画像を格納するリスト
images = []

for i in range(steps):
    tnet.forward(excite_ratio)
    y = tnet.y.reshape(-1, 1)

    image = np.zeros((n_h*n_w, 3))
    image = np.where(y, c_map, image)
    image = image.reshape(n_h, n_w, -1).astype(np.uint8)
    image = plt.imshow(image.tolist())
    images.append([image])

anim = animation.ArtistAnimation(figure, images, interval=100)
rc("animation", html="jshtml")
if save_gif:
    anim.save("torus_network2.gif", writer="pillow", fps=10)
if save_mp4:
    anim.save("torus_network2.mp4", writer="ffmpeg")
plt.close()  # 通常のグラフを非表示に
anim

結果がうまく表示されない場合は、「ランタイム」→「ランタイムを出荷時設定にリセット」を選択し、ランタイムをリセットしましょう。  
なお、計算にはGPUを利用していますが、連続で使用するとランタイムが途中で切断されることがあります。タイムステップ数を少なくするか、しばらく時間を置いてから再びお試しください。